In [1]:
# !pip install numpy
# !pip install pandas
# !pip install tqdm
# !pip install sklearn
# !pip install matplotlib

In [61]:
#This is dev branch
import datetime
import math
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing as mp
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
NUM_TIME_SLOTS = 144
NUM_DAYS_IN_DATA = 0

In [3]:
# 24 hours is divided into 144 slots where each slot is 10 mins long
def calculateTimeSlot(time,printValue=True):
    global NUM_TIME_SLOTS
    dateTime = datetime.datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
    timePart = dateTime.time()
    timeInMinutes = (timePart.hour * 60) + timePart.minute + (timePart.second/60) + 1
    timeSlot = timeInMinutes/10
    roundedTimeSlot = math.ceil(timeSlot)
    if roundedTimeSlot > NUM_TIME_SLOTS:
        roundedTimeSlot -= 1
    if printValue==True:
        print(f"time: {time} timeInMinutes: {timeInMinutes} timeSlot: {roundedTimeSlot}")
    return int(roundedTimeSlot)

def extractDayOfWeek(time):
    d = datetime.datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
    return d.weekday()

def extractDate(time):
    d = datetime.datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
    return str(d.date())

print(type(extractDate('2019-01-01 00:00:00')))

<class 'str'>


In [4]:
def extractNumberOfPOI(poiArray):
    poiCount = 0
    for poiEntry in poiArray:
        poiEntry = poiEntry + ':'
        divisions = poiEntry.split(":")
        poiCount += int(divisions[1])
    return poiCount
        

In [5]:
def readMultipleData(path,fileNamePrefix,headerNames,dataTypes,delimiter2="\t"):
    global NUM_DAYS_IN_DATA
    filesToExplore = []
    for file in os.listdir(path):
        if file.startswith(fileNamePrefix):
            filesToExplore.append(file)
            # print(f"{file} read")
        else:
            continue

    print(f"{len(filesToExplore)} files read")
    if fileNamePrefix == 'order':
        NUM_DAYS_IN_DATA = len(filesToExplore)
    
    readData = []
    for files in filesToExplore:
        fileRead = pd.read_csv(path + files, sep=delimiter2, names=headerNames,dtype=dataTypes)
        readData.append(fileRead)

    readData = pd.concat(readData, ignore_index=True)
    return readData

In [6]:
# now region Data
regionData = pd.read_csv('./training_data/cluster_map/cluster_map', sep='\t', names=['region_hash', 'region_id'],dtype={'region_hash': 'str', 'region_id': 'int'})
# print(regionData.head())
# regionData.to_csv('regionData.csv',index=False)

In [7]:
def getRegionId(regionHash):
    for _, row in regionData.iterrows():
        if row['region_hash'] == regionHash:
            return row['region_id']

In [8]:
# read order data
dataTypes = {'order_id':'str', 'driver_id':'str', 'passenger_id':'str', 'start_region_hash':'str', 'dest_region_hash':'str', 'price':'double', 'time':'str'}
orderDataPath = './training_data/order_data/'
orderData = readMultipleData(orderDataPath,'order', ['order_id', 'driver_id', 'passenger_id', 'start_region_hash', 'dest_region_hash', 'price', 'time'], dataTypes)
print("printing order data")
# print(orderData.head())


21 files read
printing order data


In [9]:
# read weather data
dataTypes={'time':'str', 'weather':'int', 'temperature':'double', 'PM2.5':'double'}
weatherDataPath = './training_data/weather_data/'
weatherData = readMultipleData(weatherDataPath,'weather', ['time', 'weather', 'temperature', 'PM2.5'], dataTypes)
print("printing weather data")
# print(weatherData.head())

21 files read
printing weather data


In [10]:
weatherData['time_slot'] = weatherData['time'].apply(calculateTimeSlot,printValue=False)
weatherData['date'] = weatherData['time'].apply(extractDate)
# weatherData['day_of_week'] = weatherData['time'].apply(extractDayOfWeek)
weatherData = weatherData.drop(['time'], axis=1)
print(weatherData.head())

   weather  temperature  PM2.5  time_slot        date
0        1          4.0  177.0          1  2016-01-01
1        1          3.0  177.0          1  2016-01-01
2        1          3.0  177.0          2  2016-01-01
3        1          3.0  177.0          2  2016-01-01
4        1          3.0  177.0          3  2016-01-01


In [11]:
orderData = pd.merge(regionData,orderData, how='inner', right_on='start_region_hash', left_on='region_hash')
print("Step 1 done")
orderData = orderData.rename(columns={'region_id': 'start_region_id'})
orderData = pd.merge(regionData,orderData, how='inner', right_on='dest_region_hash', left_on='region_hash')
orderData = orderData.rename(columns={'region_id': 'dest_region_id'})
print("Step 2 done")
orderData['time_slot'] = orderData['time'].apply(calculateTimeSlot,printValue=False)
print("Step 3 done")
orderData['day_of_week'] = orderData['time'].apply(extractDayOfWeek)
print("Step 4 done")
orderData['date'] = orderData['time'].apply(extractDate)
print("Step 5 done")
orderData = orderData.drop(['passenger_id', 'dest_region_hash','start_region_hash','price'], axis=1)
print("Step 6 done")
print(orderData.head())
# regionData=None

Step 1 done
Step 2 done
Step 3 done
Step 4 done
Step 5 done
Step 6 done
                      region_hash_x  dest_region_id   
0  90c5a34f06ac86aee0fd70e2adce7d8a               1  \
1  90c5a34f06ac86aee0fd70e2adce7d8a               1   
2  90c5a34f06ac86aee0fd70e2adce7d8a               1   
3  90c5a34f06ac86aee0fd70e2adce7d8a               1   
4  90c5a34f06ac86aee0fd70e2adce7d8a               1   

                      region_hash_y  start_region_id   
0  90c5a34f06ac86aee0fd70e2adce7d8a                1  \
1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
2  90c5a34f06ac86aee0fd70e2adce7d8a                1   
3  90c5a34f06ac86aee0fd70e2adce7d8a                1   
4  90c5a34f06ac86aee0fd70e2adce7d8a                1   

                           order_id                         driver_id   
0  1654babc363bc6d0f5d01fc0bafedc1a  cc26812d679c9e55a6bf63eed315e989  \
1  ce86a6ae5eee7a2ea954323b6c01510b  360478560b1fd4b3eb757074c91ee709   
2  b5e816c08e44565c7ed67a6f6e366708  0359

In [12]:
# orderData = pd.read_csv('orderData.csv')
print(orderData.head())
print(orderData.shape)

                      region_hash_x  dest_region_id   
0  90c5a34f06ac86aee0fd70e2adce7d8a               1  \
1  90c5a34f06ac86aee0fd70e2adce7d8a               1   
2  90c5a34f06ac86aee0fd70e2adce7d8a               1   
3  90c5a34f06ac86aee0fd70e2adce7d8a               1   
4  90c5a34f06ac86aee0fd70e2adce7d8a               1   

                      region_hash_y  start_region_id   
0  90c5a34f06ac86aee0fd70e2adce7d8a                1  \
1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
2  90c5a34f06ac86aee0fd70e2adce7d8a                1   
3  90c5a34f06ac86aee0fd70e2adce7d8a                1   
4  90c5a34f06ac86aee0fd70e2adce7d8a                1   

                           order_id                         driver_id   
0  1654babc363bc6d0f5d01fc0bafedc1a  cc26812d679c9e55a6bf63eed315e989  \
1  ce86a6ae5eee7a2ea954323b6c01510b  360478560b1fd4b3eb757074c91ee709   
2  b5e816c08e44565c7ed67a6f6e366708  0359fc335d238c6206703d1d7e3620c8   
3  25ed10b13aaa36071deecab3aa374be3  c03

In [13]:
# read POI Data
poiDataStr = {
    'region_hash':[],
    'poi_class':[]
}
with open('./training_data/poi_data/poi_data','r') as fileToRead:
    for line in fileToRead:
        line = line.strip()
        columns = line.split('\t')
        poiDataStr['region_hash'].append(columns[0])
        remData = columns[1:]
        poiDataStr['poi_class'].append(remData)
        
poiData = pd.DataFrame(poiDataStr,columns=['region_hash','poi_class'])
poiData['poi_count'] = poiData['poi_class'].apply(extractNumberOfPOI)
poiData = pd.merge(regionData,poiData, how='inner', on='region_hash')
poiData = poiData.drop(['region_hash'], axis=1)
poiData = poiData.drop(['poi_class'], axis=1)
print(poiData.head())

   region_id  poi_count
0          1     653376
1          2     343537
2          3      31125
3          4     187829
4          5      27888


In [14]:
# print(mergedDataCSV)
orderData['requests'] = 1
# print(orderData.head())
groupedMergedData = orderData.groupby(['start_region_id','time_slot','day_of_week','dest_region_id'])['requests'].agg('sum').reset_index()
# groupedMergedData = groupedMergedData.drop(['date','region_hash','order_id','driver_id','time'], axis=1)
orderData = pd.merge(orderData,groupedMergedData, how='left' , on=['start_region_id','time_slot','day_of_week','dest_region_id'])
print(orderData.head())

                      region_hash_x  dest_region_id   
0  90c5a34f06ac86aee0fd70e2adce7d8a               1  \
1  90c5a34f06ac86aee0fd70e2adce7d8a               1   
2  90c5a34f06ac86aee0fd70e2adce7d8a               1   
3  90c5a34f06ac86aee0fd70e2adce7d8a               1   
4  90c5a34f06ac86aee0fd70e2adce7d8a               1   

                      region_hash_y  start_region_id   
0  90c5a34f06ac86aee0fd70e2adce7d8a                1  \
1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
2  90c5a34f06ac86aee0fd70e2adce7d8a                1   
3  90c5a34f06ac86aee0fd70e2adce7d8a                1   
4  90c5a34f06ac86aee0fd70e2adce7d8a                1   

                           order_id                         driver_id   
0  1654babc363bc6d0f5d01fc0bafedc1a  cc26812d679c9e55a6bf63eed315e989  \
1  ce86a6ae5eee7a2ea954323b6c01510b  360478560b1fd4b3eb757074c91ee709   
2  b5e816c08e44565c7ed67a6f6e366708  0359fc335d238c6206703d1d7e3620c8   
3  25ed10b13aaa36071deecab3aa374be3  c03

In [15]:
orderData = orderData.drop(["requests_x"], axis=1)
orderData = orderData.drop(["region_hash_x"], axis=1)
orderData = orderData.rename(columns={"region_hash_y": "region_hash"})
orderData = orderData.rename(columns={"requests_y": "requests"})
print(orderData)

         dest_region_id                       region_hash  start_region_id   
0                     1  90c5a34f06ac86aee0fd70e2adce7d8a                1  \
1                     1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
2                     1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
3                     1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
4                     1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
...                 ...                               ...              ...   
7190449              66  1ecbb52d73c522f184a6fc53128b1ea1               66   
7190450              66  1ecbb52d73c522f184a6fc53128b1ea1               66   
7190451              66  1ecbb52d73c522f184a6fc53128b1ea1               66   
7190452              66  1ecbb52d73c522f184a6fc53128b1ea1               66   
7190453              66  1ecbb52d73c522f184a6fc53128b1ea1               66   

                                 order_id                      

In [16]:
orderData['temp'] = 1
orderData['answers'] = orderData['temp'].where(orderData['driver_id'].notnull(), 0)
orderData = orderData.drop(['temp'], axis=1)
groupedMergedData = orderData.groupby(['start_region_id','time_slot','day_of_week','dest_region_id'])['answers'].agg('sum').reset_index()
orderData = pd.merge(orderData,groupedMergedData, how='left' , on=['start_region_id','time_slot','day_of_week','dest_region_id'])
print(orderData)

         dest_region_id                       region_hash  start_region_id   
0                     1  90c5a34f06ac86aee0fd70e2adce7d8a                1  \
1                     1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
2                     1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
3                     1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
4                     1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
...                 ...                               ...              ...   
7190449              66  1ecbb52d73c522f184a6fc53128b1ea1               66   
7190450              66  1ecbb52d73c522f184a6fc53128b1ea1               66   
7190451              66  1ecbb52d73c522f184a6fc53128b1ea1               66   
7190452              66  1ecbb52d73c522f184a6fc53128b1ea1               66   
7190453              66  1ecbb52d73c522f184a6fc53128b1ea1               66   

                                 order_id                      

In [17]:
orderData = orderData.drop(["answers_x"], axis=1)
orderData = orderData.rename(columns={"answers_y": "answers"})

In [18]:
orderData['supply_demand'] = orderData['requests'] - orderData['answers']
print(orderData)

         dest_region_id                       region_hash  start_region_id   
0                     1  90c5a34f06ac86aee0fd70e2adce7d8a                1  \
1                     1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
2                     1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
3                     1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
4                     1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
...                 ...                               ...              ...   
7190449              66  1ecbb52d73c522f184a6fc53128b1ea1               66   
7190450              66  1ecbb52d73c522f184a6fc53128b1ea1               66   
7190451              66  1ecbb52d73c522f184a6fc53128b1ea1               66   
7190452              66  1ecbb52d73c522f184a6fc53128b1ea1               66   
7190453              66  1ecbb52d73c522f184a6fc53128b1ea1               66   

                                 order_id                      

In [19]:
orderData = orderData.drop(['order_id','driver_id','requests','answers','region_hash','time'], axis=1)
# orderData = m.drop_duplicates(subset=['region_id','time_slot','day_of_week'], keep='first')
print(orderData) # here

         dest_region_id  start_region_id  time_slot  day_of_week        date   
0                     1                1        126            4  2016-01-01  \
1                     1                1        127            4  2016-01-01   
2                     1                1        117            4  2016-01-01   
3                     1                1        124            4  2016-01-01   
4                     1                1        135            4  2016-01-01   
...                 ...              ...        ...          ...         ...   
7190449              66               66         65            3  2016-01-21   
7190450              66               66         70            3  2016-01-21   
7190451              66               66         98            3  2016-01-21   
7190452              66               66        124            3  2016-01-21   
7190453              66               66        131            3  2016-01-21   

         supply_demand  
0             

In [20]:
def getPOICount(regionId):
    for _, row in poiData.iterrows():
        if row['region_id'] == regionId:
            return row['poi_count']

In [21]:
# merge poi data here
# orderData['start_poi_count'] = orderData['region_id'].apply(getPOICount)
# orderData['dest_poi_count'] = orderData['dest_region_id'].apply(getPOICount)

orderData = pd.merge(orderData,poiData, how="inner", left_on='start_region_id', right_on='region_id')
orderData = orderData.rename(columns={"poi_count": "start_poi_count"})
orderData = pd.merge(orderData,poiData, how="inner", left_on='dest_region_id',right_on='region_id')
orderData = orderData.rename(columns={"poi_count": "dest_poi_count"})
orderData = orderData.drop(['region_id_x','region_id_y'], axis=1)
print(orderData)

         dest_region_id  start_region_id  time_slot  day_of_week        date   
0                     1                1        126            4  2016-01-01  \
1                     1                1        127            4  2016-01-01   
2                     1                1        117            4  2016-01-01   
3                     1                1        124            4  2016-01-01   
4                     1                1        135            4  2016-01-01   
...                 ...              ...        ...          ...         ...   
7190449              66               66         65            3  2016-01-21   
7190450              66               66         70            3  2016-01-21   
7190451              66               66         98            3  2016-01-21   
7190452              66               66        124            3  2016-01-21   
7190453              66               66        131            3  2016-01-21   

         supply_demand  start_poi_count

In [22]:
# orderData['date'] = orderData['date'].apply(lambda x: str(x))
# weatherData['date'] = weatherData['date'].apply(lambda x: str(x))
# print(weatherData['date'][0]==orderData['date'][0])
# print(orderData['date'][0])

In [23]:
# merge weather data here
orderData = pd.merge(weatherData,orderData, how="inner", on=['date','time_slot'])
print(orderData)

          weather  temperature  PM2.5  time_slot        date  dest_region_id   
0               1          4.0  177.0          1  2016-01-01               1  \
1               1          4.0  177.0          1  2016-01-01               1   
2               1          4.0  177.0          1  2016-01-01               1   
3               1          4.0  177.0          1  2016-01-01               1   
4               1          4.0  177.0          1  2016-01-01               1   
...           ...          ...    ...        ...         ...             ...   
10274734        2          1.0   59.0        144  2016-01-21              65   
10274735        2          1.0   59.0        144  2016-01-21              66   
10274736        2          1.0   59.0        144  2016-01-21              66   
10274737        2          1.0   59.0        144  2016-01-21              66   
10274738        2          1.0   59.0        144  2016-01-21              66   

          start_region_id  day_of_week 

In [24]:
m = orderData.drop_duplicates(keep='first')
print(m)

          weather  temperature  PM2.5  time_slot        date  dest_region_id   
0               1          4.0  177.0          1  2016-01-01               1  \
77              1          4.0  177.0          1  2016-01-01               1   
78              1          4.0  177.0          1  2016-01-01               1   
79              1          4.0  177.0          1  2016-01-01               1   
80              1          4.0  177.0          1  2016-01-01               1   
...           ...          ...    ...        ...         ...             ...   
10273506        2          1.0   59.0        144  2016-01-21              62   
10273508        2          1.0   59.0        144  2016-01-21              62   
10273509        2          1.0   59.0        144  2016-01-21              64   
10273514        2          1.0   59.0        144  2016-01-21              65   
10273516        2          1.0   59.0        144  2016-01-21              66   

          start_region_id  day_of_week 

<h1 style="background-color:green">Train Model</h1>

In [110]:
# Apply model here
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

X = m[['start_region_id','time_slot','day_of_week','dest_region_id','start_poi_count','dest_poi_count','weather','temperature','PM2.5']]
Y = m['supply_demand']

# RandomForestRegressor
# Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y, test_size=0.1, random_state=42)
# AImodel = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
# AImodel.fit(Xtrain,Ytrain)

# polynomial regression
# polinomial_features = PolynomialFeatures(degree=10)
# Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y, test_size=0.1, random_state=42)
# AImodel = LinearRegression()
# AImodel.fit(Xtrain,Ytrain)

# DecisionTreeRegressor
# Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y, test_size=0.05, random_state=42)
# AImodel = DecisionTreeRegressor(max_depth=200)
# AImodel.fit(Xtrain,Ytrain)

# gradientBoostingRegressor
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.1, random_state=42)
AImodel = GradientBoostingRegressor(n_estimators=100, max_depth=10, learning_rate=1)
AImodel.fit(Xtrain,Ytrain)


GradientBoostingRegressor(learning_rate=5, max_depth=8)

In [111]:
print("Accuracy",AImodel.score(Xtest,Ytest)) # score on test split

Accuracy -1.2611972705882345e+120


In [57]:
# make rolling window function
def rollTheWindow(data,windowSize):
    datasetLength = len(data)
    listOfPredictions = []
    numberOfWindows = datasetLength//windowSize
    index = 0
    lastWindow = False
    for i in range(numberOfWindows):
        rollingWindow = []
        lowerLimit = index
        upperLimit = index+windowSize
        if upperLimit > datasetLength:
            upperLimit = datasetLength-1
        if lowerLimit >= upperLimit:
            break
        rollingWindow.append(data[lowerLimit:upperLimit])
        rollingWindow = pd.concat(rollingWindow)
        X_WINDOW_INPUT = rollingWindow[['start_region_id','time_slot','day_of_week','dest_region_id','start_poi_count','dest_poi_count','weather','temperature','PM2.5']]
        Y_WINDOW_OUTPUT = rollingWindow['supply_demand']
        currentPrediction = AImodel.predict(X_WINDOW_INPUT)
        meanSqError = mean_squared_error(Y_WINDOW_OUTPUT, currentPrediction)
        print(f"Prediction for window [{lowerLimit},{upperLimit}] MSE={meanSqError}")
        predictionTuple = (lowerLimit,upperLimit,currentPrediction,meanSqError,Y_WINDOW_OUTPUT)
        listOfPredictions.append(predictionTuple)
        index = upperLimit + 1
        if index >= datasetLength-1 and lastWindow == False:
            lastWindow = True
            index = datasetLength-1
        elif index >= datasetLength-1 and lastWindow == True:
            break
    return listOfPredictions

In [103]:
listOfPredictions =  rollTheWindow(orderData,10000)
maxError= max(listOfPredictions, key=lambda x: x[3])
minEror = min(listOfPredictions, key=lambda x: x[3])
print(f"Max error is {maxError[3]} for window {maxError[0]} to {maxError[1]}")
print(f"Min error is {minEror[3]} for window {minEror[0]} to {minEror[1]}")

Prediction for window [0,10000] MSE=3.9862081019128754
Prediction for window [10001,20001] MSE=17.28785948149057
Prediction for window [20002,30002] MSE=27.398301318433333
Prediction for window [30003,40003] MSE=29.463790115256398
Prediction for window [40004,50004] MSE=173.2384351512804
Prediction for window [50005,60005] MSE=167.60911690566925
Prediction for window [60006,70006] MSE=15.42792764628004
Prediction for window [70007,80007] MSE=97.77556294219215
Prediction for window [80008,90008] MSE=41.71029813561038
Prediction for window [90009,100009] MSE=26.445341498562506
Prediction for window [100010,110010] MSE=24.61864730090458
Prediction for window [110011,120011] MSE=12.86125157092026
Prediction for window [120012,130012] MSE=14.042146937310672
Prediction for window [130013,140013] MSE=14.800297605631236
Prediction for window [140014,150014] MSE=8.61427600173667
Prediction for window [150015,160015] MSE=8.239562820784323
Prediction for window [160016,170016] MSE=12.835414600571

In [29]:
print("Accuracy",AImodel.score(X,Y)) # score on whole data

Accuracy 0.9985111533069524


<h1 style="background-color:blue">Read Test Data</h1>

In [30]:
# read test data
# read region Data
regionTestData = pd.read_csv('./test_set/cluster_map/cluster_map', sep='\t', names=['region_hash', 'region_id'],dtype={'region_hash': 'str', 'region_id': 'int'})
print(regionTestData.head())
#regionData.to_csv('regionData.csv',index=False)

                        region_hash  region_id
0  90c5a34f06ac86aee0fd70e2adce7d8a          1
1  f2c8c4bb99e6377d21de71275afd6cd2          2
2  58c7a4888306d8ff3a641d1c0feccbe3          3
3  b26a240205c852804ff8758628c0a86a          4
4  4b9e4cf2fbdc8281b8a1f9f12b80ce4d          5


In [31]:
# read order data
dataTypes = {'order_id':'str', 'driver_id':'str','start_region_hash':'str', 'dest_region_hash':'str' ,'time':'str'}
orderDataPath = './test_set/order_data/'
orderTestData = readMultipleData(orderDataPath,'test_order', ['order_id', 'driver_id', 'start_region_hash','dest_region_hash','time'], dataTypes,delimiter2=",")
print("printing order data")
print(orderTestData.head())

5 files read
printing order data
                           order_id                         driver_id   
0  a903b5f7f65f1dc7f4ee94fec74673be  dce1e90fc91ed39a7b04a22d02910a7d  \
1  b7c838beaf12a2132776a1f00e016038  1f39eba5ce330f20c95177826a122e12   
2  0e0d3c93298ed59281352e34c6f1ec5a  8e4e2bc0342b3c55edea2723f6613e36   
3  1f6d0d7f68f216c4333969d6152a0a8b  5a33393e516673c8e9a065915667a30f   
4  bcf6050d9f5b270f8beb3ff80b01b435  f0232b8e45abc5ca92ca0f90fa811e7c   

                  start_region_hash                  dest_region_hash   
0  d4ec2125aff74eded207d2d915ef682f  d4ec2125aff74eded207d2d915ef682f  \
1  2407d482f0ffa22a947068f2551fe62c  2407d482f0ffa22a947068f2551fe62c   
2  b26a240205c852804ff8758628c0a86a  3a43dcdff3c0b66b1acb1644ff055f9d   
3  4725c39a5e5f4c188d382da3910b3f3f  4725c39a5e5f4c188d382da3910b3f3f   
4  dd8d3b9665536d6e05b29c2648c0e69a  a5609739c6b5c2719a3752327c5e33a7   

                  time  
0  2016-01-23 07:20:52  
1  2016-01-23 13:08:17  
2  2016-01-23 

In [32]:
# read weather data
dataTypes={'time':'str', 'weather':'int', 'temperature':'double', 'PM2.5':'double'}
weatherTestDataPath = './test_set/weather_data/'
weatherTestData = readMultipleData(weatherTestDataPath,'weather_data', ['time', 'weather', 'temperature', 'PM2.5'], dataTypes)
print("printing weather data")
print(weatherTestData.head())

5 files read
printing weather data
                  time  weather  temperature  PM2.5
0  2016-01-23 07:01:12        4          1.0   94.0
1  2016-01-23 11:14:04        3         -1.0  107.0
2  2016-01-23 13:16:44        3         -1.0   56.0
3  2016-01-23 15:02:02        3          0.0   48.0
4  2016-01-23 15:16:39        3          0.0   48.0


In [33]:
#pre processing weather Test Data based on weather timeSlot 
weatherTestData['time_slot'] = weatherTestData['time'].apply(calculateTimeSlot,printValue=False)
weatherTestData['date'] = weatherTestData['time'].apply(extractDate)
# weatherData['day_of_week'] = weatherData['time'].apply(extractDayOfWeek)
weatherTestData = weatherTestData.drop(['time'], axis=1)
print(weatherTestData.head())

   weather  temperature  PM2.5  time_slot        date
0        4          1.0   94.0         43  2016-01-23
1        3         -1.0  107.0         68  2016-01-23
2        3         -1.0   56.0         80  2016-01-23
3        3          0.0   48.0         91  2016-01-23
4        3          0.0   48.0         92  2016-01-23


In [34]:
orderTestData = pd.merge(regionTestData,orderTestData, how='inner', right_on='start_region_hash', left_on='region_hash')
print("Step 1 done")
orderTestData = orderTestData.rename(columns={'region_id': 'start_region_id'})
orderTestData = pd.merge(regionTestData,orderTestData, how='inner', right_on='dest_region_hash', left_on='region_hash')
orderTestData = orderTestData.rename(columns={'region_id': 'dest_region_id'})
print("Step 2 done")
orderTestData['time_slot'] = orderTestData['time'].apply(calculateTimeSlot,printValue=False)
print("Step 3 done")
orderTestData['day_of_week'] = orderTestData['time'].apply(extractDayOfWeek)
print("Step 4 done")
orderTestData['date'] = orderTestData['time'].apply(extractDate)
print("Step 5 done")
orderTestData = orderTestData.drop(['dest_region_hash','start_region_hash'], axis=1)
print("Step 6 done")
print(orderTestData.head())

Step 1 done
Step 2 done
Step 3 done
Step 4 done
Step 5 done
Step 6 done
                      region_hash_x  dest_region_id   
0  90c5a34f06ac86aee0fd70e2adce7d8a               1  \
1  90c5a34f06ac86aee0fd70e2adce7d8a               1   
2  90c5a34f06ac86aee0fd70e2adce7d8a               1   
3  90c5a34f06ac86aee0fd70e2adce7d8a               1   
4  90c5a34f06ac86aee0fd70e2adce7d8a               1   

                      region_hash_y  start_region_id   
0  90c5a34f06ac86aee0fd70e2adce7d8a                1  \
1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
2  90c5a34f06ac86aee0fd70e2adce7d8a                1   
3  90c5a34f06ac86aee0fd70e2adce7d8a                1   
4  90c5a34f06ac86aee0fd70e2adce7d8a                1   

                           order_id                         driver_id   
0  31bcca3b90b4d971697156b63900d514  172b54135139345fb9d24d0b23635387  \
1  cca7aca7134cffb46957d59897dcdbbc  773dd49e4148d807b099b42372fd5ac7   
2  85cbab6eaea4b6bf1bc2cd7ce6701a93  75eb

In [35]:
# read POI Data
poiDataStr = {
    'region_hash':[],
    'poi_class':[]
}
with open('./test_set/poi_data/poi_data','r') as fileToRead:
    for line in fileToRead:
        line = line.strip()
        columns = line.split('\t')
        poiDataStr['region_hash'].append(columns[0])
        remData = columns[1:]
        poiDataStr['poi_class'].append(remData)
        
poiTestData = pd.DataFrame(poiDataStr,columns=['region_hash','poi_class'])
poiTestData['poi_count'] = poiTestData['poi_class'].apply(extractNumberOfPOI)
poiTestData = pd.merge(regionTestData,poiTestData, how='inner', on='region_hash')
poiTestData = poiTestData.drop(['region_hash'], axis=1)
poiTestData = poiTestData.drop(['poi_class'], axis=1)
print(poiTestData.head())

   region_id  poi_count
0          1     653376
1          2     343537
2          3      31125
3          4     187829
4          5      27888


In [36]:
# print(mergedDataCSV)
orderTestData['requests'] = 1
# print(orderData.head())
groupedTestMergedData = orderTestData.groupby(['start_region_id','time_slot','day_of_week','dest_region_id'])['requests'].agg('sum').reset_index()
# groupedMergedData = groupedMergedData.drop(['date','region_hash','order_id','driver_id','time'], axis=1)
orderTestData = pd.merge(orderTestData,groupedTestMergedData, how='left' , on=['start_region_id','time_slot','day_of_week','dest_region_id'])
print(orderTestData.head())

                      region_hash_x  dest_region_id   
0  90c5a34f06ac86aee0fd70e2adce7d8a               1  \
1  90c5a34f06ac86aee0fd70e2adce7d8a               1   
2  90c5a34f06ac86aee0fd70e2adce7d8a               1   
3  90c5a34f06ac86aee0fd70e2adce7d8a               1   
4  90c5a34f06ac86aee0fd70e2adce7d8a               1   

                      region_hash_y  start_region_id   
0  90c5a34f06ac86aee0fd70e2adce7d8a                1  \
1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
2  90c5a34f06ac86aee0fd70e2adce7d8a                1   
3  90c5a34f06ac86aee0fd70e2adce7d8a                1   
4  90c5a34f06ac86aee0fd70e2adce7d8a                1   

                           order_id                         driver_id   
0  31bcca3b90b4d971697156b63900d514  172b54135139345fb9d24d0b23635387  \
1  cca7aca7134cffb46957d59897dcdbbc  773dd49e4148d807b099b42372fd5ac7   
2  85cbab6eaea4b6bf1bc2cd7ce6701a93  75eb96caaa6bacb30b810996d655dd54   
3  50f3588c595eca21e6be6182d521decc  c7b

In [37]:
orderTestData = orderTestData.drop(["requests_x"], axis=1)
orderTestData = orderTestData.drop(["region_hash_x"], axis=1)
orderTestData = orderTestData.rename(columns={"region_hash_y": "region_hash"})
orderTestData = orderTestData.rename(columns={"requests_y": "requests"})
print(orderTestData.head())

   dest_region_id                       region_hash  start_region_id   
0               1  90c5a34f06ac86aee0fd70e2adce7d8a                1  \
1               1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
2               1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
3               1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
4               1  90c5a34f06ac86aee0fd70e2adce7d8a                1   

                           order_id                         driver_id   
0  31bcca3b90b4d971697156b63900d514  172b54135139345fb9d24d0b23635387  \
1  cca7aca7134cffb46957d59897dcdbbc  773dd49e4148d807b099b42372fd5ac7   
2  85cbab6eaea4b6bf1bc2cd7ce6701a93  75eb96caaa6bacb30b810996d655dd54   
3  50f3588c595eca21e6be6182d521decc  c7bddad06fdf4fd7bb77c3007118d215   
4  3c751c264452c4d73e90dd6066566e94  54402bda533bb92ec3d697c361ca683d   

                  time  time_slot  day_of_week        date  requests  
0  2016-01-23 19:09:39        116            5  2016-01-2

In [38]:
orderTestData['temp'] = 1
orderTestData['answers'] = orderTestData['temp'].where(orderTestData['driver_id'].notnull(), 0)
orderTestData = orderTestData.drop(['temp'], axis=1)
groupedTestMergedData = orderTestData.groupby(['start_region_id','time_slot','day_of_week','dest_region_id'])['answers'].agg('sum').reset_index()
orderTestData = pd.merge(orderTestData,groupedTestMergedData, how='left' , on=['start_region_id','time_slot','day_of_week','dest_region_id'])
print(orderTestData)

        dest_region_id                       region_hash  start_region_id   
0                    1  90c5a34f06ac86aee0fd70e2adce7d8a                1  \
1                    1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
2                    1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
3                    1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
4                    1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
...                ...                               ...              ...   
467906              66  1ecbb52d73c522f184a6fc53128b1ea1               66   
467907              66  1ecbb52d73c522f184a6fc53128b1ea1               66   
467908              66  1ecbb52d73c522f184a6fc53128b1ea1               66   
467909              66  1ecbb52d73c522f184a6fc53128b1ea1               66   
467910              66  1ecbb52d73c522f184a6fc53128b1ea1               66   

                                order_id                         driver_id 

In [39]:
orderTestData = orderTestData.drop(["answers_x"], axis=1)
orderTestData = orderTestData.rename(columns={"answers_y": "answers"})
# orderTestData.to_csv('mergedOrderData.csv',index=False)

In [40]:
# orderTestData['supply_demand'] = orderTestData['requests'] - orderTestData['answers']
orderTestData['supply_demand'] = orderTestData['requests']
print(orderTestData)

        dest_region_id                       region_hash  start_region_id   
0                    1  90c5a34f06ac86aee0fd70e2adce7d8a                1  \
1                    1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
2                    1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
3                    1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
4                    1  90c5a34f06ac86aee0fd70e2adce7d8a                1   
...                ...                               ...              ...   
467906              66  1ecbb52d73c522f184a6fc53128b1ea1               66   
467907              66  1ecbb52d73c522f184a6fc53128b1ea1               66   
467908              66  1ecbb52d73c522f184a6fc53128b1ea1               66   
467909              66  1ecbb52d73c522f184a6fc53128b1ea1               66   
467910              66  1ecbb52d73c522f184a6fc53128b1ea1               66   

                                order_id                         driver_id 

In [41]:
orderTestData = orderTestData.drop(['order_id','driver_id','requests','answers','region_hash','time'], axis=1)
print(orderTestData)

        dest_region_id  start_region_id  time_slot  day_of_week        date   
0                    1                1        116            5  2016-01-23  \
1                    1                1        127            5  2016-01-23   
2                    1                1         43            5  2016-01-23   
3                    1                1         81            5  2016-01-23   
4                    1                1         93            5  2016-01-23   
...                ...              ...        ...          ...         ...   
467906              66               66         69            6  2016-01-31   
467907              66               66        104            6  2016-01-31   
467908              66               66        103            6  2016-01-31   
467909              66               66         57            6  2016-01-31   
467910              66               66         92            6  2016-01-31   

        supply_demand  
0                 130  
1  

In [42]:
orderTestData = pd.merge(orderTestData,poiData, how="inner", left_on='start_region_id', right_on='region_id')
orderTestData = orderTestData.rename(columns={"poi_count": "start_poi_count"})
orderTestData = pd.merge(orderTestData,poiData, how="inner", left_on='dest_region_id',right_on='region_id')
orderTestData = orderTestData.rename(columns={"poi_count": "dest_poi_count"})
orderTestData = orderTestData.drop(['region_id_x','region_id_y'], axis=1)
print(orderTestData)

        dest_region_id  start_region_id  time_slot  day_of_week        date   
0                    1                1        116            5  2016-01-23  \
1                    1                1        127            5  2016-01-23   
2                    1                1         43            5  2016-01-23   
3                    1                1         81            5  2016-01-23   
4                    1                1         93            5  2016-01-23   
...                ...              ...        ...          ...         ...   
467906              63               63         79            2  2016-01-27   
467907              63               63         56            2  2016-01-27   
467908              63               63         68            4  2016-01-29   
467909              63               63         68            4  2016-01-29   
467910              63               63         57            4  2016-01-29   

        supply_demand  start_poi_count  dest_poi_co

In [43]:
orderTestData = pd.merge(weatherTestData,orderTestData, how="inner", on=['date','time_slot'])
print(orderTestData)


        weather  temperature  PM2.5  time_slot        date  dest_region_id   
0             4          1.0   94.0         43  2016-01-23               1  \
1             4          1.0   94.0         43  2016-01-23               1   
2             4          1.0   94.0         43  2016-01-23               1   
3             4          1.0   94.0         43  2016-01-23               1   
4             4          1.0   94.0         43  2016-01-23               1   
...         ...          ...    ...        ...         ...             ...   
288877        8          1.0   69.0        141  2016-01-31              49   
288878        8          1.0   69.0        141  2016-01-31              64   
288879        8          1.0   69.0        141  2016-01-31              64   
288880        8          1.0   69.0        141  2016-01-31              32   
288881        8          1.0   69.0        141  2016-01-31              17   

        start_region_id  day_of_week  supply_demand  start_poi_

In [44]:
mTest = orderTestData.drop_duplicates(keep='first')
print(mTest)

        weather  temperature  PM2.5  time_slot        date  dest_region_id   
0             4          1.0   94.0         43  2016-01-23               1  \
34            4          1.0   94.0         43  2016-01-23               1   
43            4          1.0   94.0         43  2016-01-23               1   
55            4          1.0   94.0         43  2016-01-23               1   
56            4          1.0   94.0         43  2016-01-23               1   
...         ...          ...    ...        ...         ...             ...   
288876        8          1.0   69.0        141  2016-01-31              13   
288877        8          1.0   69.0        141  2016-01-31              49   
288878        8          1.0   69.0        141  2016-01-31              64   
288880        8          1.0   69.0        141  2016-01-31              32   
288881        8          1.0   69.0        141  2016-01-31              17   

        start_region_id  day_of_week  supply_demand  start_poi_

<h1 style="background-color:skyblue; color:black">Prediction on Test Data</h1>

In [45]:
# XTest = mTest[['region_id','time_slot','day_of_week','weather','poi_count']]
# YTest = mTest['supply_demand']
# print("accuracy: ",AImodel.score(XTest,YTest))
# print(YTest)

In [104]:
listOfPredictions =  rollTheWindow(orderTestData,100)
maxError= max(listOfPredictions, key=lambda x: x[3])
minEror = min(listOfPredictions, key=lambda x: x[3])
print(f"Max error is {maxError[3]} for window {maxError[0]} to {maxError[1]}")
print(f"Min error is {minEror[3]} for window {minEror[0]} to {minEror[1]}")

Prediction for window [0,100] MSE=359.78718920165704
Prediction for window [101,201] MSE=103.50396317750489
Prediction for window [202,302] MSE=65.30162928654883
Prediction for window [303,403] MSE=84.64288549246983
Prediction for window [404,504] MSE=29.6747818086653
Prediction for window [505,605] MSE=20.13864533796675
Prediction for window [606,706] MSE=169.42248301483477
Prediction for window [707,807] MSE=499.24295642279685
Prediction for window [808,908] MSE=6027.298812636492
Prediction for window [909,1009] MSE=2047.7255423670126
Prediction for window [1010,1110] MSE=263.89740120188867
Prediction for window [1111,1211] MSE=163.29246983724082
Prediction for window [1212,1312] MSE=689.7833237165574
Prediction for window [1313,1413] MSE=646.7192028580189
Prediction for window [1414,1514] MSE=151.401646361028
Prediction for window [1515,1615] MSE=1104.9449684454075
Prediction for window [1616,1716] MSE=107.20872307015823
Prediction for window [1717,1817] MSE=420.9005522453184
Predic

In [105]:
# print mean of mean squared error
errors = [x[3] for x in listOfPredictions]
print("Mean of mean squared error for above predictions is: ",np.mean(errors))

Mean of mean squared error for above predictions is:  3490.846998622751


In [106]:
Xtest = orderTestData[['start_region_id','time_slot','day_of_week','dest_region_id','start_poi_count','dest_poi_count','weather','temperature','PM2.5']]
Ytest = orderTestData['supply_demand']
print("Accuracy of the model is: ",AImodel.score(Xtest,Ytest))

Accuracy of the model is:  0.5484061672744255
